In [42]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import seaborn as sns
from collections import Counter

Summary : 
    Define the sucess in order to categorize it : here 0 or 1 depending on a threshold (over 50,60,.. of popularity)
    Perform a GNB (for continous variables)
    Define the correctness : here since there are just a few 1 --> we define the accuracy by the number of right 1 
    problem : not enough one to learn 
    
 

# BAYES 

# Preprocessing

In [72]:
df=pd.read_csv("data.csv")
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('defore.csv')
dfX=df[['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence','year']]
dfy=df['popularity']

In [73]:
binarizer = Binarizer(threshold=50, copy=True)
df['popularity']=binarizer.fit_transform(df['popularity'].values.reshape(df.shape[0], 1))
#recup la colonne artist puis set, et mapping
#calculer la précision sur les data qui valent 1
X=dfX.to_numpy()
y=dfy.to_numpy()
xTrain,xTest,yTrain,yTest=train_test_split(X,y,test_size=0.50,random_state=0)

# Oversampling

In [74]:
#oversampling 
print('Original data shape %s' % Counter(yTrain))
sm = SMOTE()
xTrain, yTrain = sm.fit_resample(xTrain, yTrain)
print('Resampled data shape %s' % Counter(yTrain))

Original data shape Counter({0: 67034, 1: 17920})
Resampled data shape Counter({0: 67034, 1: 67034})


# Artists

In [85]:
dfA=pd.read_csv("data_by_artist.csv")
dfA=dfA[['artists','popularity']]
print(dfA.head())

                                             artists  popularity
0                   "Cats" 1981 Original London Cast   38.000000
1                          "Cats" 1983 Broadway Cast   33.076923
2        "Fiddler On The Roof” Motion Picture Chorus   34.285714
3     "Fiddler On The Roof” Motion Picture Orchestra   34.444444
4  "Joseph And The Amazing Technicolor Dreamcoat"...   42.555556


# Differenet Bayes

**BAYES with Scikit-lear**
*first try with a gaussian distribution: fit continuous data*
*NB : Gaussian Distribution, not pertinant
*sur-apprentissage ?*

In [75]:
def GNB(xTrain,yTrain,xTest,yTest):
    gnb=GaussianNB()
    yPred=gnb.fit(xTrain,yTrain).predict(xTest)
    print("Number of mislabeled points out of a total %d points : %d"% (xTest.shape[0], (yTest != yPred).sum()))     
    print(metrics.accuracy_score(yTest, yPred))
    return yPred
yPred=GNB(xTrain,yTrain,xTest,yTest)
#sns.jointplot(x=yTest, y=yPred, kind="kde", color="r")

Number of mislabeled points out of a total 84955 points : 20917
0.7537872991583779


**Right prediction on 1-values**

In [76]:
def OneCheck(yTest,yPred):
    yTest1,yPred1=[],[]  
    right1=0
    for i,test in enumerate(yTest):
        if test==1:
            yTest1.append(test)
            yPred1.append(yPred[i])
            if yPred[i]==1:
                right1+=1
    print("Right Prediction for sucess",right1/len(yTest1))
OneCheck(yTest,yPred)

Right Prediction for sucess 0.8511539748020961


What value of Popularity ? 

In [82]:

for pop in range(45,95,5):
    print(pop)
    df=pd.read_csv("data.csv")
    df = df.sample(frac=1).reset_index(drop=True)
    dfX=df[['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence','year']]
    dfy=df['popularity']
    binarizer = Binarizer(threshold=pop, copy=True)
    df['popularity']=binarizer.fit_transform(df['popularity'].values.reshape(df.shape[0], 1))
    #recup la colonne artist puis set, et mapping
    #calculer la précision sur les data qui valent 1
    X=dfX.to_numpy()
    y=dfy.to_numpy()
    xTrain,xTest,yTrain,yTest=train_test_split(X,y,test_size=0.50,random_state=0)
    sm = SMOTE()
    xTrain, yTrain = sm.fit_resample(xTrain, yTrain)
    GNB(xTrain,yTrain,xTest,yTest)
    OneCheck(yTest,yPred)

45
Number of mislabeled points out of a total 84955 points : 18968
0.7767288564534165
Right Prediction for sucess 0.390860674065943
50
Number of mislabeled points out of a total 84955 points : 20809
0.755058560414337
Right Prediction for sucess 0.39213382566856886
55
Number of mislabeled points out of a total 84955 points : 21892
0.7423106350420811
Right Prediction for sucess 0.3921488946684005
60
Number of mislabeled points out of a total 84955 points : 23032
0.7288917662291802
Right Prediction for sucess 0.3957719410634209
65
Number of mislabeled points out of a total 84955 points : 23083
0.7282914484138662
Right Prediction for sucess 0.3834621691262591
70
Number of mislabeled points out of a total 84955 points : 23702
0.7210052380672121
Right Prediction for sucess 0.3923117430226435
75
Number of mislabeled points out of a total 84955 points : 22158
0.7391795656524042
Right Prediction for sucess 0.387140902872777
80
Number of mislabeled points out of a total 84955 points : 15496
0.81

**with scaling**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols=['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence','year']
def scaleColumns(df, cols2Scale):
    for col in cols2Scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df

dfScaled=scaleColumns(df,cols)
X=dfScaled[cols].to_numpy()
y=dfy.to_numpy()
xTrain,xTest,yTrain,yTest=train_test_split(X,y,test_size=0.25,random_state=0)
gnb=GaussianNB()
yPred=gnb.fit(xTrain,yTrain).predict(xTest)
print("Number of mislabeled points out of a total %d points : %d"% (xTest.shape[0], (yTest != yPred).sum()))
metrics.accuracy_score(yTest, yPred)*100


**with discretization & binarization**

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
#first n bins and then one hot encoder
#discrete : mode
#let's do an ordinalEncoder : since variable are ordinal and not nominal 
#(ordered categories but the distance is not known)


disc = KBinsDiscretizer(n_bins=4, encode='uniform', strategy='uniform')
columns=df.columns.values.tolist()
cols=['acousticness','danceability', 'duration_ms', 'energy','instrumentalness',  'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

def Discretize(df,cols):
    long=df.shape[0]
    pass


def Binarize(df,cols):
    long=df.shape[0]
    disc = KBinsDiscretizer(n_bins=3, encode='onehot', strategy='uniform')
    for c in cols:
        mean=df[c].mean()
        name=c+'Bin'
        binarizer = Binarizer(threshold=mean, copy=True)
        df[name]=binarizer.fit_transform(df[c].values.reshape(long, 1))
    return df


dfB=Binarize(df,cols)
print(dfB.head())

dfBX=dfB[['acousticnessBin','danceabilityBin','energyBin','instrumentalnessBin','livenessBin','loudnessBin','speechinessBin','tempoBin','valenceBin']][:1000]
#dfDX=dfD[['acousticnessDisc','danceabilityDisc','energyDisc','instrumentalnessDisc','livenessDisc','loudnessDisc','speechinessDisc','tempoDisc','valenceDisc']]
dfBy=df['popularity'][:1000]
XB=dfBX.to_numpy()
yB=dfBy.to_numpy()
xBTrain,xBTest,yBTrain,yBTest=train_test_split(XB,yB,test_size=0.50,random_state=0)


   acousticness                                            artists  \
0     -0.045706                  ['Steve Wariner', 'Garth Brooks']   
1     -1.299841                                     ['Humble Pie']   
2      1.284527  ['Johann Sebastian Bach', 'Amsterdam Toonkunst...   
3     -0.744011                                     ['Tom Browne']   
4     -0.831631                                      ['Aerosmith']   

   danceability  duration_ms    energy  explicit                      id  \
0      1.544668    -0.850460  0.218434         0  4bjKZrfowKmAfzHrRhpfcQ   
1     -1.506455    -0.386899  0.173556         0  6KL9kMRMmevqkJNnaVOeCJ   
2     -2.282067    -0.910911 -0.551979         0  2PbDTYjEz0AWyNiumrr22n   
3      0.575153    -0.080498  0.248353         0  3Yahrb4joLsIIdUSqyHJZ4   
4     -1.152867     0.538460  1.811618         0  2pg89WdhaUgJzVX3YobOht   

   instrumentalness  key  liveness  ...  acousticnessBin  danceabilityBin  \
0         -0.523507    9 -0.919650  ...      

**Bayes with SKlearn & binarized values**

In [6]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
yBPred=clf.fit(xBTrain, yBTrain).predict(xBTest)
print("Number of mislabeled points out of a total %d points : %d"% (xBTest.shape[0], (yBTest != yBPred).sum()))
metrics.accuracy_score(yBTest, yBPred)*100

Number of mislabeled points out of a total 500 points : 1


99.8

In [7]:
#apprenticeship issues : too many data not varied 